In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from multi_env import make_reversi_vec_env, SelfPlayEnv
import torch as th
from players import RandomPlayer
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import numpy as np

In [3]:
board_shape = 8
n_envs = 10
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

# Modificación de librería para que haga argmax solo sobre las válidas

In [4]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
)

In [5]:
model.predict(env.reset())

(array([11, 41,  4, 34, 42, 56, 34, 31, 49, 52]), None)

# Custom ActorCriticPolicy 

https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/policies.py

In [6]:
from boardgame2 import ReversiEnv

In [7]:
env_not_vect = ReversiEnv(board_shape)

In [8]:
(board, player) = env_not_vect.reset()

In [9]:
env_not_vect.get_valid((board, player))

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [10]:
def get_actions_mask(state):
    player = 1
    valid_actions = env_not_vect.get_valid((state, player))
    return valid_actions.reshape(-1)  


In [11]:
get_actions_mask(env.reset()[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [12]:
def get_batch_actions_mask(states_batch):
    batch_size = states_batch.shape[0]
    masks = np.array([get_actions_mask(s[0]) for s in states_batch])
    return masks.reshape(batch_size, 1, -1)

In [13]:
from typing import Tuple

In [14]:
class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(
        self,
        *args, # Todos los argumentos posicionales de ActorCriticPolicy
        actions_mask_func=None, # El nuevo argumento
        **kwargs # Todos los argumentos opcionales de ActorCriticPolicy
    ):
        super(CustomActorCriticPolicy, self).__init__(
            *args,
            **kwargs
        )
        if actions_mask_func:
            self.get_actions_mask = actions_mask_func
    
    
    
    def sample_masked_actions(self, obs, distribution, deterministic=False, return_distribution=False):
        # Dada las obs y distribuciones luego de evaluar la red neuronal, samplear solo las acciones válidas
        # Las obs se usan para que con self.get_actions_mask se obtengan las acciones válidas
        # las distribuciones son el resultado de evaluar la red neuronal y van a dar acciones no validas
        # Generar una nueva distribución (del lado de los logits preferentemente) donde las acciones no válidas
        # tengan probabildad nula de ser muestreadas
        # Luego se modifican abajo los métodos
        # _predict, forward y evaluate_actions
        # Si tiene el flag de return_distribution en true devuelve la distribución nueva
        # Caso contrario devuelve las acciones
        # Para tener en cuenta, obs tiene dimensión [batch_size, channels, H, W]
        # Recomendamos poner un print(obs.shape)
        # y correr:
        # obs = env.reset()
        # actions, _ = model.predict(obs)
        # Para sacarse las dudas
        
        masks = 1 - self.get_actions_mask(obs)
        masks = -masks * (10**8) # Numero negativo grande en los moviemientos invalidos
        
        if th.is_tensor(distribution.logits):
            masks = th.from_numpy(masks).to(model.device)
        
        masked_logits = distribution.logits + masks.reshape(distribution.logits.shape[0], distribution.logits.shape[1])
        if return_distribution:
            return th.distributions.Categorical(logits=masked_logits)
        if deterministic:
            return th.argmax(masked_logits, axis=1)
        return th.distributions.Categorical(logits=masked_logits).sample()
    
    def _predict(self, observation, deterministic=False):
        """
        Get the action according to the policy for a given observation.
        :param observation:
        :param deterministic: Whether to use stochastic or deterministic actions
        :return: Taken action according to the policy
        """
        latent_pi, _, latent_sde = self._get_latent(observation)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde)
        
        if self.get_actions_mask:
            actions = self.sample_masked_actions(observation, distribution.distribution, deterministic=deterministic)
        else:
            actions = distribution.get_actions(deterministic=deterministic)
        
        return actions
    
    def forward(self, obs: th.Tensor, deterministic: bool = False):
        """
        Forward pass in all the networks (actor and critic)
        :param obs: Observation
        :param deterministic: Whether to sample or use deterministic actions
        :return: action, value and log probability of the action
        """
        latent_pi, latent_vf, latent_sde = self._get_latent(obs)
        # Evaluate the values for the given observations
        values = self.value_net(latent_vf)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde=latent_sde)
        
        
        if self.get_actions_mask:
            actions = self.sample_masked_actions(obs, distribution.distribution, deterministic=deterministic)
        else:
            actions = distribution.get_actions(deterministic=deterministic)

        log_prob = distribution.log_prob(actions)
        return actions, values, log_prob
    
    def evaluate_actions(self, obs: th.Tensor, actions: th.Tensor) -> Tuple[th.Tensor, th.Tensor, th.Tensor]:
        """
        Evaluate actions according to the current policy,
        given the observations.
        :param obs:
        :param actions:
        :return: estimated value, log likelihood of taking those actions
            and entropy of the action distribution.
        """
        latent_pi, latent_vf, latent_sde = self._get_latent(obs)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde)
        distrib = self.sample_masked_actions(obs, distribution.distribution, return_distribution=True)

        log_prob = distrib.log_prob(actions)
        values = self.value_net(latent_vf)
        return values, log_prob, distrib.entropy()

In [15]:
model = PPO(
    CustomActorCriticPolicy,
    env,
    verbose=0,
    policy_kwargs = {'actions_mask_func': get_batch_actions_mask}
)


In [16]:
# Testeo de predict
model.policy.get_actions_mask(env.reset())

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
    

In [17]:
obs = env.reset()
actions, _ = model.predict(obs)

In [18]:
# Verificar que las acciones son válidas
actions

array([42, 29, 21, 29, 29, 20, 37, 20, 29, 29])

In [19]:
model.device

device(type='cpu')

In [20]:
# Testeo de forward
model.policy(th.from_numpy(obs).to(model.device))

(tensor([44, 20, 21, 43, 43, 20, 21, 29, 34, 43]),
 tensor([[-0.1765],
         [ 0.2399],
         [-0.6131],
         [ 0.2399],
         [ 0.2399],
         [ 0.2399],
         [-0.6131],
         [ 0.2399],
         [ 0.2399],
         [ 0.2399]], grad_fn=<AddmmBackward>),
 tensor([-4.1564, -4.1552, -4.1615, -4.1576, -4.1576, -4.1552, -4.1615, -4.1652,
         -4.1555, -4.1576], grad_fn=<SqueezeBackward1>))

# Corremos PPO

In [21]:
board_shape = 8
n_envs = 6
gamma = 0.99
ent_coef = 0.0
gae_lambda = 0.95
n_epochs = 10

In [22]:
prefix = 'Reversi_PPO'
suffix = 'masked_actions'
model_name = f'{prefix}_{board_shape}by{board_shape}_{gamma}_{gae_lambda}_{ent_coef}_{n_epochs}_{n_envs}_{suffix}'
best_model_save_path = f'./models/{model_name}'
print(model_name)
print(best_model_save_path)

Reversi_PPO_8by8_0.99_0.95_0.0_10_6_masked_actions
./models/Reversi_PPO_8by8_0.99_0.95_0.0_10_6_masked_actions


In [23]:
model = PPO(
    CustomActorCriticPolicy,
    env,
    verbose=0,
    tensorboard_log='tensorboard_log',
    gamma=gamma,
    gae_lambda=gae_lambda,
    ent_coef=ent_coef,
    n_epochs=n_epochs,
    policy_kwargs = {'actions_mask_func': get_batch_actions_mask}
)

In [24]:
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

In [25]:
# El entorno de evaluación no corre en paralelo por eso uno solo
eval_env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=1,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

In [26]:
eval_callback = EvalCallback(
    eval_env = eval_env,
    eval_freq=1_000,
    n_eval_episodes=500,
    deterministic=True,
    verbose=1,
    best_model_save_path=best_model_save_path,
) 

In [28]:
model.learn(total_timesteps=int(1e5), callback=[eval_callback])

TypeError: can't pickle _thread.lock objects